# Celebrity Recognition using Amazon Rekognition


This notebook provides a walkthrough of [celebrity recognition API](https://docs.aws.amazon.com/rekognition/latest/dg/celebrities.html) in Amazon Rekognition. You can quickly identify well known people in your video and image libraries to catalog footage and photos for marketing, advertising, and media industry use cases.

* https://docs.aws.amazon.com/rekognition/latest/dg/celebrities.html
* https://docs.aws.amazon.com/rekognition/latest/dg/API_RecognizeCelebrities.html
* https://docs.aws.amazon.com/rekognition/latest/dg/API_StartCelebrityRecognition.html
* https://docs.aws.amazon.com/rekognition/latest/dg/API_GetCelebrityRecognition.html

# Setup

In [1]:
import boto3
from IPython.display import HTML, display, Image as IImage
from PIL import Image, ImageDraw, ImageFont
import time
import os

In [2]:
import sagemaker
import boto3

sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()
bucket = sagemaker_session.default_bucket()
region = boto3.Session().region_name

In [3]:
rekognition = boto3.client("rekognition")
s3 = boto3.client("s3")

In [4]:
!mkdir -p ./tmp
temp_folder = "tmp/"

# Recognize Celebrities in Images

In [5]:
imageName = "content-moderation/media/GrandTourjc.png"

In [6]:
display(IImage(url=s3.generate_presigned_url("get_object", Params={"Bucket": bucket, "Key": imageName})))

# Call Amazon Rekognition to recognize celebrities in the image
https://docs.aws.amazon.com/rekognition/latest/dg/API_RecognizeCelebrities.html

In [7]:
recognizeCelebritiesResponse = rekognition.recognize_celebrities(
    Image={
        "S3Object": {
            "Bucket": bucket,
            "Name": imageName,
        }
    }
)

AccessDeniedException: An error occurred (AccessDeniedException) when calling the RecognizeCelebrities operation: User: arn:aws:sts::298039562326:assumed-role/mod-6a56f92ff726428b-SageMakerExecutionRole-131HJ8VVEU80K/SageMaker is not authorized to perform: rekognition:RecognizeCelebrities

# Review JSON response returned by Rekognition Celebrity Recognition API
In the JSON response below, you will see CelebrityFaces which contains information about recognized celebrities.

For each recognized celebrity, you will see information like Name, Id, Urls and additional information about their facial attributes.

In [ ]:
display(recognizeCelebritiesResponse)

# Show Image with Bounding Boxes Around Recognized Celebrities

In [ ]:
def drawBoundingBoxes(sourceImage, boxes):
    # blue, green, red, grey
    colors = ((255, 255, 255), (255, 255, 255), (76, 182, 252), (52, 194, 123))

    # Download image locally
    imageLocation = temp_folder + os.path.basename(sourceImage)
    s3.download_file(bucket, sourceImage, imageLocation)

    # Draws BB on Image
    bbImage = Image.open(imageLocation)
    draw = ImageDraw.Draw(bbImage)
    width, height = bbImage.size
    col = 0
    maxcol = len(colors)
    line = 3
    for box in boxes:
        x1 = int(box[1]["Left"] * width)
        y1 = int(box[1]["Top"] * height)
        x2 = int(box[1]["Left"] * width + box[1]["Width"] * width)
        y2 = int(box[1]["Top"] * height + box[1]["Height"] * height)

        draw.text((x1, y1), box[0], colors[col])
        for l in range(line):
            draw.rectangle((x1 - l, y1 - l, x2 + l, y2 + l), outline=colors[col])
        col = (col + 1) % maxcol

    imageFormat = "PNG"
    ext = sourceImage.lower()
    if ext.endswith("jpg") or ext.endswith("jpeg"):
        imageFormat = "JPEG"

    bbImage.save(imageLocation, format=imageFormat)

    display(bbImage)

In [ ]:
boxes = []
celebrities = recognizeCelebritiesResponse["CelebrityFaces"]
for celebrity in celebrities:
    boxes.append((celebrity["Name"], celebrity["Face"]["BoundingBox"]))

drawBoundingBoxes(imageName, boxes)

# Recognize Celebrities in Videos
 Celebrity recognition in video is an async operation. 
 https://docs.aws.amazon.com/rekognition/latest/dg/API_StartCelebrityRecognition.html
 - We first start celebrity recognition job which returns a Job Id.
 - We can then call `get_celebrity_recognition` to get the job status and after job is complete, we can get celebrity metadata.
 - In production use cases, you would usually use StepFucntion or SNS topic to get notified when job is complete.

In [ ]:
videoName = "content-moderation/media/GrandTour720.mp4"
strDetail = "Celebrites detected in video<br>=======================================<br>"
strOverall = "Celebrities in the overall video:<br>=======================================<br>"

In [ ]:
s3FilePrefix = "https://s3.amazonaws.com"
if not region == "us-east-1":
    s3FilePrefix = "https://s3-{}.amazonaws.com".format(region)

s3VideoUrl = "{0}/{1}/{2}".format(s3FilePrefix, bucket, videoName)

videoTag = "<video controls='controls' autoplay width='640' height='360' name='Video' src='{0}'></video>".format(
    s3VideoUrl
)

videoui = "<table><tr><td style='vertical-align: top'>{}</td></tr><tr><td>{}</td></tr></table>".format(
    videoTag, strDetail
)

display(HTML(videoui))

# Call Rekognition to Start a Job for Celebrity Rekognition

In [ ]:
startCelebrityRekognition = rekognition.start_celebrity_recognition(
    Video={
        "S3Object": {
            "Bucket": bucket,
            "Name": videoName,
        }
    },
)

celebrityJobId = startCelebrityRekognition["JobId"]
display("Job Id: {0}".format(celebrityJobId))

# Wait for Celebrity Rekognition Job to Complete
* In production use cases, you would usually use StepFucntion or SNS topic to get notified when job is complete.


In [ ]:
%%time

getCelebrityRecognition = rekognition.get_celebrity_recognition(JobId=celebrityJobId, SortBy="TIMESTAMP")

while getCelebrityRecognition["JobStatus"] == "IN_PROGRESS":
    time.sleep(5)
    print(".", end="")

    getCelebrityRecognition = rekognition.get_celebrity_recognition(JobId=celebrityJobId, SortBy="TIMESTAMP")

display(getCelebrityRecognition["JobStatus"])

# Review JSON response returned by Rekognition Celebrity Recognition API
In the JSON response below, you will see list Celebrities which contains information about recognized celebrities.

For each recognized celebrity, you will see information like Timestamp, Name, Id, Urls
 and additional information about their facial attributes.

In [ ]:
display(getCelebrityRecognition)

# Display Names of Recognized Celebrities in the Video
Display timestamps and celebrites detected at that time.


In [ ]:
theCelebs = {}

# Celebrities detected in each frame
for celebrity in getCelebrityRecognition["Celebrities"]:
    if "Celebrity" in celebrity:
        cconfidence = celebrity["Celebrity"]["Confidence"]
        if cconfidence > 95:
            ts = celebrity["Timestamp"]
            cname = celebrity["Celebrity"]["Name"]
            strDetail = strDetail + "At {} ms: {} (Confidence: {})<br>".format(ts, cname, round(cconfidence, 2))
            if not cname in theCelebs:
                theCelebs[cname] = cname


# Unique faces detected in video
for theCeleb in theCelebs:
    strOverall = strOverall + "Name: {}<br>".format(theCeleb)

# Display results
display(HTML(strOverall))

# Detect Non-Celebrities
Now let us try an image with non-celebrities in the image.


In [ ]:
customCelebrityImageName = "content-moderation/media/chris-antje.png"

In [ ]:
display(
    IImage(url=s3.generate_presigned_url("get_object", Params={"Bucket": bucket, "Key": customCelebrityImageName}))
)

In [ ]:
# Call Amazon Rekognition to recognize celebrities in the image

customCelebrityResponse = rekognition.recognize_celebrities(
    Image={
        "S3Object": {
            "Bucket": bucket,
            "Name": customCelebrityImageName,
        }
    }
)

# Display Rekognition Response
You will see Rekognition return an empty list for CelebrityFaces and 

UnrecognizedFaces list with unrecognized faces that were detected in the image.

In the next module you will learn how to get custom-celebrity faces recognized.


In [ ]:
display(customCelebrityResponse)

# Show Image and Bounded Boxes around Detected Faces

In [ ]:
cboxes = []
faces = customCelebrityResponse["UnrecognizedFaces"]
for face in faces:
    cboxes.append(("Unrecognized Face", face["BoundingBox"]))

drawBoundingBoxes(customCelebrityImageName, cboxes)

# Congratulations!
You have successfully used Amazon Rekognition to identify celebrities in images an videos. 

# References
- https://docs.aws.amazon.com/rekognition/latest/dg/celebrities.html
- https://docs.aws.amazon.com/rekognition/latest/dg/API_RecognizeCelebrities.html
- https://docs.aws.amazon.com/rekognition/latest/dg/API_StartCelebrityRecognition.html
- https://docs.aws.amazon.com/rekognition/latest/dg/API_GetCelebrityRecognition.html

# Release Resources

In [ ]:
%%html

<p><b>Shutting down your kernel for this notebook to release resources.</b></p>
<button class="sm-command-button" data-commandlinker-command="kernelmenu:shutdown" style="display:none;">Shutdown Kernel</button>
        
<script>
try {
    els = document.getElementsByClassName("sm-command-button");
    els[0].click();
}
catch(err) {
    // NoOp
}    
</script>

In [ ]:
%%javascript

try {
    Jupyter.notebook.save_checkpoint();
    Jupyter.notebook.session.delete();
}
catch(err) {
    // NoOp
}